# Table of contents

1. [Importing Libraries & Loading Data](#Importing)
2. [Cleaning & Preparing Data](#Clean&Prep)
    1. [Renaming columns](#Renaming)
    2. [Dropping bad data](#DropBadData)
    3. [Resetting values & types](#ResetValues)
3. [Populating Null Values](#PopulatingNulls)
4. [Adding Metrics & Features](#AddingMetrics)

In [1]:
#### TO DO IN FUTURE ####
## 1) Improve BsmtQual fill model for Nulls in test data
## 2) Add new features (lot area??) ## Would need to get creative on building new one ##
## 3) Remove low impact features as I add higher impact one
## 4) Better clean the data...
## 5) Figure out a way to be smarter to weight inputs into meta-features (ie. TotalArea)
#### TO DO IN FUTURE ####

# 1) Importing Libraries & Loading Data<a name="Importing"></a>

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

from IPython.display import display
pd.options.display.max_columns = None

%matplotlib inline

In [3]:
train = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')

# 2) Cleaning & Preparing Data<a name="Clean&Prep"></a>

### Renaming columns<a name="Renaming"></a>

In [4]:
## Function to replace spaces in the column titles
def remove_space(df):
    col_no_space = []
    for col in list(df.columns):
        col_no_space.append(col.replace(' ',''))
    return col_no_space

## Removing spaces from column titles in both test sets
train.columns = remove_space(train)
test.columns = remove_space(test)

### Dropping bad data<a name="DropBadData"></a>

In [5]:
## Dropping all columns from train data where more than 40% of the data is missing
train.drop(columns=['Alley','PoolQC','Fence','MiscFeature','FireplaceQu'], inplace=True)

### Resetting values & types<a name="ResetValues"></a>

In [6]:
## Converting CentralAir to binary
train['CentralAir'] = [1 if x =='Y' else 0 for x in train['CentralAir']]
test['CentralAir'] = [1 if x =='Y' else 0 for x in test['CentralAir']]

## Set MSSubClass to a series of strings (categorical)
train['MSSubClass'] = train['MSSubClass'].astype(str)
test['MSSubClass'] = test['MSSubClass'].astype(str)

In [7]:
## Sets dictionary to recategorize quality as numeric value (higher = better)
recat_dict = {
    'Ex' : 5,
    'Gd' : 4,
    'TA' : 3,
    'Fa' : 2,
    'Po' : 1,
    'Na' : 0,
    'GLQ' : 6,
    'ALQ' : 5,
    'BLQ' : 4,
    'Rec' : 3,
    'LwQ' : 2,
    'Unf' : 1,
    'NA' : 0,
    'Av' : 3,
    'Mn' : 2,
    'No' : 1,
    'Fin' : 3,
    'RFn' : 2
    
}

## Replaces quality indices with numeric values
train.replace(to_replace=recat_dict, inplace=True)
test.replace(to_replace=recat_dict, inplace=True)

## Changes these types to a float
train[['ExterQual','BsmtQual','KitchenQual','GarageQual','BsmtFinType1','BsmtFinType2',
      'HeatingQC','GarageCond','BsmtExposure']] = \
train[['ExterQual','BsmtQual','KitchenQual','GarageQual','BsmtFinType1','BsmtFinType2',
      'HeatingQC','GarageCond','BsmtExposure']].astype(float)

test[['ExterQual','BsmtQual','KitchenQual','GarageQual','BsmtFinType1','BsmtFinType2',
      'HeatingQC','GarageCond','BsmtExposure']] = \
test[['ExterQual','BsmtQual','KitchenQual','GarageQual','BsmtFinType1','BsmtFinType2',
      'HeatingQC','GarageCond','BsmtExposure']].astype(float)

### NOTE: I tested a few of these beforehand to make sure the numeric scales had higher correlations than
### any of the dummy variables. I find that using numbers make sense and the correlations seem
### stronger than creating dummy variables for Quality

# 3) Populating null values<a name="PopulatingNulls"></a>

#### Test Data: Basement Quality
In the test data, basement quality (BsmtQual) had a number of Null or NaN values. Since the feature is paramount to the final model, a sub-model (EDA below) was used to fill in these Nulls.

In [8]:
## Setting up X and y for filling in Basement Quality on test data
bsmt_qual_filter = ['BsmtQual','YearBuilt','OverallQual','ExterQual','KitchenQual']
bq_df = test[bsmt_qual_filter].dropna()
y_test_bq = bq_df['BsmtQual']
X_test_bq = bq_df[bsmt_qual_filter].drop(columns=['BsmtQual'])

In [9]:
## Instantiating ridge model and standard scaler
ridge = RidgeCV(cv=5)
SS = StandardScaler()

## Scaling the X data
X_test_bq_s = SS.fit_transform(X_test_bq)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [10]:
## Fitting the model
ridge.fit(X_test_bq_s, y_test_bq)
X_test_bq_full = test[bsmt_qual_filter].drop(columns=['BsmtQual'])
X_test_bq_full_s = SS.transform(X_test_bq_full)

## Predicting Basement Quality
test['BsmtQualPreds'] = ridge.predict(X_test_bq_full_s)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [11]:
## Filling Test BsmtQual Nulls with predictions
test['BsmtQual'].fillna(value=test['BsmtQualPreds'], inplace=True)

# 4) Adding Metrics & Features<a name="AddingMetrics"></a>

In [12]:
## Defining new metric for how old the house is
train['HouseAge'] = 2019 - train['YearBuilt']
test['HouseAge'] = 2019 - test['YearBuilt']

#### This ended up not being used.

In [13]:
## Defining new metric for the total area of the house based on combination of individual house areas
#### NOTE: Weighted according to strength of correlation.
#### NOTE: TotalArea is not actually in square feet anymore

train['TotalArea'] = (0.628925 * train['TotalBsmtSF']) + (0.697038 * train['GrLivArea'])\
+ (0.650270*train['GarageArea']) - (0.045328*train['LowQualFinSF'])

test['TotalArea'] = (0.628925 * test['TotalBsmtSF']) + (0.697038 * test['GrLivArea'])\
+ (0.650270*test['GarageArea']) - (0.045328*test['LowQualFinSF'])

In [14]:
## Defining new metric for total Quality
#### NOTE: Weighted according to strength of correlation.

train['QualMetric'] = (0.692336*train['KitchenQual']) + (0.678307*train['BsmtQual']) + \
(0.712146*train['ExterQual']) + (0.800207*train['OverallQual'])

test['QualMetric'] = (0.692336*test['KitchenQual']) + (0.678307*test['BsmtQual']) + \
(0.712146*test['ExterQual']) + (0.800207*test['OverallQual'])

In [15]:
## Defining new metric for Total Baths
#### NOTE: Half baths are multiplied by... wait for it... half
#### This intuitively makes sense and seems to drive stronger correlations

train['TotalBaths'] = train['BsmtFullBath'] + train['FullBath'] + \
(0.5*train['HalfBath']) + (0.5*train['BsmtHalfBath'])

test['TotalBaths'] = test['BsmtFullBath'] + test['FullBath'] + \
(0.5*test['HalfBath']) + (0.5*test['BsmtHalfBath'])

In [16]:
## Defining new metric for total Porch or Deck Area
#### NOTE: Weighting by correlations didn't add value here.

train['PorchArea'] = train['ScreenPorch'] + train['3SsnPorch']\
+ train['OpenPorchSF'] + train['EnclosedPorch'] + train['WoodDeckSF']

test['PorchArea'] = test['ScreenPorch'] + test['3SsnPorch']\
+ test['OpenPorchSF'] + test['EnclosedPorch'] + test['WoodDeckSF']

In [17]:
## Defining function to find and return DataFrame with only the dummy variables that
## have a correlation > 0.15

def find_corr_dummies(df,col,target):
    temp_dumms_corr = pd.get_dummies(df[[col,target]]).corr()[target]
    temp_dumms_corr = list(temp_dumms_corr[abs(temp_dumms_corr) > .15].index.drop(target))
    return pd.get_dummies(df)[temp_dumms_corr]

#### FUTURE CODE: While this works well with train data, where target is known, it doesn't work so hot
#### with the test data. I'd have this return a list of column names in the future so it's more widely usable

In [18]:
## Adding Neighborhoods with strong correlations to my train data
train = pd.concat([train,find_corr_dummies(train,'Neighborhood','SalePrice')], axis=1)

## Adding Neighborhoods with strong correlations to my test data
#### As addressed above, I couldn't apply find_corr_dummies to the test data (or use its output)
test = pd.concat([test, 
                  pd.get_dummies(test)[['Neighborhood_Edwards', 'Neighborhood_IDOTRR', 'Neighborhood_NAmes',
        'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown',
        'Neighborhood_Somerst', 'Neighborhood_StoneBr']]],
                 axis=1)

In [19]:
## Adding strong correlations for MSSubClass
#### NOTE: I never added this to test because the correlations were insignificant
train = pd.concat([train,find_corr_dummies(train,'MSSubClass','SalePrice')], axis=1)

In [20]:
## Adding a variable for the mansions found in the data that are cheap for their other features
## 2 mansions were identified that drove high MSE and lay way outside the expectations
train['mansion'] = [1 if x > 5000 else 0 for x in train['TotalArea']]
test['mansion'] = [1 if x > 5000 else 0 for x in test['TotalArea']]

In [21]:
## EDA Identified a skew in the target, so log is used to make target more normal
train['SalePriceLog'] = np.log(train['SalePrice'])

In [22]:
## Exporting data
train.to_csv('../datasets/exports/train_clean.csv', index=False)
test.to_csv('../datasets/exports/test_clean.csv', index=False)